In [1]:
# Read the csv dataset
import py_entitymatching as em
A = em.read_csv_metadata('../datasets/yelp_data.csv')
A['ID'] = range(0, len(A))
em.set_key(A, 'ID')

B = em.read_csv_metadata('../datasets/inspection_data.csv')
B['ID'] = range(0, len(B))
em.set_key(B, 'ID')



Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


True

In [2]:
A.head()

,name,address,zipcode,cuisine,ID
0,barn joo 35,"34 w 35th st, ,",10001,korean gastropubs tapas/small plates,0
1,friedman's,"132 w 31st st, ,",10001,american (new) gluten-free breakfast & brunch,1
2,juniper,"237 w 35th st,",10001,cocktail bars american (new),2
3,haymaker bar and kitchen,"252 w 29th st, ,",10001,gastropubs beer bar,3
4,izakaya mew,"53 w 35th st, basement,",10001,sushi bars izakaya cocktail bars,4


In [3]:
# Down-sample
sample_A, sample_B = em.down_sample(A, B, size=18000, y_param=2)
# Save the datasets because it changes every time we perform down-sampling.
em.to_csv_metadata(sample_A, '../datasets/sample_A_generated.csv')
em.to_csv_metadata(sample_B, '../datasets/sample_B_generated.csv')

0%                          100%
[##############################] | ETA: 00:00:34 | ETA: 00:00:34 | ETA: 00:00:31 | ETA: 00:00:30 | ETA: 00:00:28 | ETA: 00:00:27 | ETA: 00:00:26 | ETA: 00:00:24 | ETA: 00:00:23 | ETA: 00:00:22 | ETA: 00:00:21 | ETA: 00:00:19 | ETA: 00:00:18 | ETA: 00:00:17 | ETA: 00:00:16 | ETA: 00:00:15 | ETA: 00:00:14 | ETA: 00:00:13 | ETA: 00:00:12 | ETA: 00:00:10 | ETA: 00:00:09 | ETA: 00:00:08 | ETA: 00:00:07 | ETA: 00:00:06 | ETA: 00:00:05 | ETA: 00:00:04 | ETA: 00:00:03 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:32


In [4]:
# Perform blocking
ab = em.AttrEquivalenceBlocker()
C = ab.block_tables(sample_A, sample_B, 'zipcode', 'zipcode', l_output_attrs=['name', 'address', 'zipcode', 'cuisine'], r_output_attrs=['name', 'address', 'zipcode', 'cuisine'])
ob = em.OverlapBlocker()
D = ob.block_candset(C, 'address', 'address', overlap_size=4)

0%                          100%
[##############################] | ETA: 00:00:49 | ETA: 00:00:46 | ETA: 00:00:43 | ETA: 00:00:41 | ETA: 00:00:39 | ETA: 00:00:38 | ETA: 00:00:36 | ETA: 00:00:34 | ETA: 00:00:33 | ETA: 00:00:31 | ETA: 00:00:30 | ETA: 00:00:28 | ETA: 00:00:26 | ETA: 00:00:25 | ETA: 00:00:23 | ETA: 00:00:22 | ETA: 00:00:21 | ETA: 00:00:19 | ETA: 00:00:17 | ETA: 00:00:16 | ETA: 00:00:14 | ETA: 00:00:13 | ETA: 00:00:11 | ETA: 00:00:09 | ETA: 00:00:08 | ETA: 00:00:06 | ETA: 00:00:04 | ETA: 00:00:03 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:48


In [5]:
# Describe the candidate set
print(len(D))
D.head(50)

1428


,_id,ltable_ID,rtable_ID,ltable_name,ltable_address,ltable_zipcode,ltable_cuisine,rtable_name,rtable_address,rtable_zipcode,rtable_cuisine
240,240,0,19502,barn joo 35,"34 w 35th st, ,",10001,korean gastropubs tapas/small plates,barn joo 35,34 w 35th st,10001,korean
628,628,1,19256,friedman's,"132 w 31st st, ,",10001,american (new) gluten-free breakfast & brunch,dee daa,132 w 31st st,10001,thai
1152,1152,3,18952,haymaker bar and kitchen,"252 w 29th st, ,",10001,gastropubs beer bar,haymaker bar & kitchen,252 w 29th st,10001,american
1323,1323,4,1922,izakaya mew,"53 w 35th st, basement,",10001,sushi bars izakaya cocktail bars,mew,53 w 35th st,10001,japanese
2682,2682,8,611,the maze,"32 w 32nd st, fl 3,",10001,korean cocktail bars mexican,the kunjip,32 w 32nd st,10001,korean
3030,3030,9,14910,woorijip authentic korean food,"12 w 32nd st, ,",10001,korean,gong cha,12 w 32nd st,10001,cafã©/coffee/tea
3250,3250,10,5334,jongro bbq,"22 w 32nd st, ,",10001,korean barbeque beer bar,jongro bbq,22 w 32nd st,10001,korean
3662,3662,11,13617,five senses,"9 w 32nd st, ,",10001,korean barbeque,gammeeok,9 w 32nd st,10001,korean
3687,3687,11,1905,five senses,"9 w 32nd st, ,",10001,korean barbeque,5 senses,9 w 32nd st,10001,korean
6393,6393,20,13406,smithfield,"138 w 25th st, ,",10001,pubs american (traditional) sports bars,smithfield hall,138 w 25th st,10001,american


In [6]:
# Sample the candidate set.
S = em.sample_table(D, sample_size=1000) # C is the candidate set to be sampled from.
# Write the sample set to a file for further processing.
em.to_csv_metadata(S, '../datasets/candidate_sample_generated.csv')

File already exists at ../datasets/candidate_sample_generated.csv; Overwriting it
Metadata file already exists at ../datasets/candidate_sample_generated.metadata. Overwriting it


True

In [7]:
S

,_id,ltable_ID,rtable_ID,ltable_name,ltable_address,ltable_zipcode,ltable_cuisine,rtable_name,rtable_address,rtable_zipcode,rtable_cuisine
240,240,0,19502,barn joo 35,"34 w 35th st, ,",10001,korean gastropubs tapas/small plates,barn joo 35,34 w 35th st,10001,korean
628,628,1,19256,friedman's,"132 w 31st st, ,",10001,american (new) gluten-free breakfast & brunch,dee daa,132 w 31st st,10001,thai
1152,1152,3,18952,haymaker bar and kitchen,"252 w 29th st, ,",10001,gastropubs beer bar,haymaker bar & kitchen,252 w 29th st,10001,american
1323,1323,4,1922,izakaya mew,"53 w 35th st, basement,",10001,sushi bars izakaya cocktail bars,mew,53 w 35th st,10001,japanese
2682,2682,8,611,the maze,"32 w 32nd st, fl 3,",10001,korean cocktail bars mexican,the kunjip,32 w 32nd st,10001,korean
3030,3030,9,14910,woorijip authentic korean food,"12 w 32nd st, ,",10001,korean,gong cha,12 w 32nd st,10001,cafã©/coffee/tea
3250,3250,10,5334,jongro bbq,"22 w 32nd st, ,",10001,korean barbeque beer bar,jongro bbq,22 w 32nd st,10001,korean
6393,6393,20,13406,smithfield,"138 w 25th st, ,",10001,pubs american (traditional) sports bars,smithfield hall,138 w 25th st,10001,american
6721,6721,21,154,abiko curry,"2 w 32nd st,",10001,japanese,ddm yupdduk,2 w 32nd st,10001,korean
6763,6763,21,15005,abiko curry,"2 w 32nd st,",10001,japanese,paik's noodle,2 w 32nd st,10001,korean


## Feature extraction

In [31]:
match_f = em.get_features_for_matching(A, B)
G = em.read_csv_metadata('../datasets/candidate_sample_labeled.csv')
em.copy_properties(S, G)
em.get_fk_ltable(G)
#atypes = em.get_attr_types(G)
H = em.extract_feature_vecs(S, feature_table=match_f)
catalog = em.get_catalog()

Metadata file is not present in the given path; proceeding to read the csv file.
0%                          100%
[##############################] | ETA: 00:00:06 | ETA: 00:00:06 | ETA: 00:00:05 | ETA: 00:00:05 | ETA: 00:00:05 | ETA: 00:00:05 | ETA: 00:00:05 | ETA: 00:00:05 | ETA: 00:00:05 | ETA: 00:00:04 | ETA: 00:00:05 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:08


In [32]:
H

,_id,ltable_ID,rtable_ID,name_name_jac_qgm_3_qgm_3,name_name_cos_dlm_dc0_dlm_dc0,name_name_jac_dlm_dc0_dlm_dc0,name_name_mel,name_name_lev_dist,name_name_lev_sim,name_name_nmw,...,cuisine_cuisine_jac_dlm_dc0_dlm_dc0,cuisine_cuisine_mel,cuisine_cuisine_lev_dist,cuisine_cuisine_lev_sim,cuisine_cuisine_nmw,cuisine_cuisine_sw,ID_ID_exm,ID_ID_anm,ID_ID_lev_dist,ID_ID_lev_sim
240,240,0,19502,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,11.0,...,0.250000,0.833333,30.0,0.166667,-24.0,6.0,0,0.000000,4,0.200000
628,628,1,19256,0.000000,0.000000,0.000000,0.465079,8.0,0.200000,-1.0,...,0.000000,0.438889,43.0,0.044444,-39.0,1.0,0,0.000052,4,0.200000
1152,1152,3,18952,0.724138,0.750000,0.600000,0.927814,3.0,0.875000,19.0,...,0.000000,0.451754,15.0,0.210526,-7.0,3.0,0,0.000158,5,0.000000
1323,1323,4,1922,0.200000,0.707107,0.500000,0.000000,8.0,0.272727,-5.0,...,0.000000,0.378472,30.0,0.062500,-22.0,2.0,0,0.002081,4,0.000000
2682,2682,8,611,0.222222,0.500000,0.333333,0.780000,6.0,0.400000,2.0,...,0.250000,0.842857,22.0,0.214286,-16.0,6.0,0,0.013093,3,0.000000
3030,3030,9,14910,0.000000,0.000000,0.000000,0.511111,26.0,0.133333,-18.0,...,0.000000,0.395833,14.0,0.125000,-8.0,2.0,0,0.000604,4,0.200000
3250,3250,10,5334,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,10.0,...,0.250000,0.850000,18.0,0.250000,-12.0,6.0,0,0.001875,4,0.000000
6393,6393,20,13406,0.526316,0.707107,0.500000,0.933333,5.0,0.666667,5.0,...,0.200000,0.735043,31.0,0.205128,-23.0,8.0,0,0.001492,4,0.200000
6721,6721,21,154,0.000000,0.000000,0.000000,0.454545,11.0,0.000000,0.0,...,0.000000,0.458333,7.0,0.125000,-1.0,2.0,0,0.136364,3,0.000000
6763,6763,21,15005,0.000000,0.000000,0.000000,0.546387,10.0,0.230769,1.0,...,0.000000,0.458333,7.0,0.125000,-1.0,2.0,0,0.001400,5,0.000000
